##第六周作业1：实验使用不同的RNN结构，实现一个人脸图像分类器。至少对比2种以上结构训练损失和准确率差异，如：LSTM、GRU、RNN、BiRNN等。要求使用tensorboard，提交代码及run目录和可视化截图。
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html

###LSTM

In [20]:
from sklearn.datasets import fetch_olivetti_faces
from torchvision.transforms.v2 import ToTensor     # 转换图像数据为张量
import torch as torch

In [21]:
import torch.nn as nn
from torch.utils.data import DataLoader

In [22]:
class RNN_Classifier(nn.Module):
    def __init__(self,):
        super().__init__()
        self.rnn=nn.LSTM(
            input_size=64,
            hidden_size=128,
            bias=True,
            num_layers=3,
            batch_first=True
        )
        self.fc=nn.Linear(128,40)
    
    def forward(self,x):
        outputs,l_h=self.rnn(x)

        out=self.fc(outputs[:,-1,:])
        return out
    

In [23]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
olivetti_faces = fetch_olivetti_faces(data_home='./face_data', shuffle=True)
print(olivetti_faces.data.shape)
print(olivetti_faces.target.shape)
print(olivetti_faces.images.shape)

(400, 4096)
(400,)
(400, 64, 64)


In [25]:
len(set(olivetti_faces.target))

40

In [26]:
##images = torch.tensor(olivetti_faces.data)
##targets = torch.tensor(olivetti_faces.target,dtype=torch.long)

In [27]:
###划分训练数据和测试数据
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(olivetti_faces.images, olivetti_faces.target, 
                                                     train_size=0.80, 
                                                    random_state=100, shuffle=True, stratify=olivetti_faces.target)

 

##参数stratify=olivetti_faces.target 表示安装target分层抽样



In [28]:
# 定义超参数
LR = 1e-3   ##学习率
epochs = 80      ##训练次数
BATCH_SIZE = 15  ###批次

In [29]:
dataset = [(img,lbl) for img,lbl in zip(X_train, y_train)]
print(dataset[0])

train_dl = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)  # shuffle=True表示打乱数据

len(dataset)

(array([[0.5082645 , 0.6280992 , 0.6694215 , ..., 0.57024795, 0.57438016,
        0.5371901 ],
       [0.59090906, 0.6694215 , 0.5       , ..., 0.5661157 , 0.57024795,
        0.57438016],
       [0.56198347, 0.6280992 , 0.46694216, ..., 0.57024795, 0.58677685,
        0.5785124 ],
       ...,
       [0.16115703, 0.14876033, 0.1446281 , ..., 0.11157025, 0.11157025,
        0.11570248],
       [0.16115703, 0.15289256, 0.14876033, ..., 0.10743801, 0.10743801,
        0.11983471],
       [0.14876033, 0.13636364, 0.1446281 , ..., 0.11157025, 0.11983471,
        0.12396694]], shape=(64, 64), dtype=float32), np.int64(31))


320

In [30]:
###测试集
dataset_test = [(img,lbl) for img,lbl in zip(X_test, y_test)]
print(dataset_test[0])

test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)  # shuffle=True表示打乱数据

len(test_loader)

(array([[0.5247934 , 0.5371901 , 0.5785124 , ..., 0.61157024, 0.59504133,
        0.55785125],
       [0.5165289 , 0.5206612 , 0.55785125, ..., 0.607438  , 0.58264464,
        0.553719  ],
       [0.5041322 , 0.5041322 , 0.5289256 , ..., 0.59090906, 0.57438016,
        0.553719  ],
       ...,
       [0.34710744, 0.34710744, 0.34710744, ..., 0.53305787, 0.5247934 ,
        0.446281  ],
       [0.3429752 , 0.3429752 , 0.3429752 , ..., 0.53305787, 0.5247934 ,
        0.42975205],
       [0.338843  , 0.338843  , 0.338843  , ..., 0.5289256 , 0.5082645 ,
        0.43801653]], shape=(64, 64), dtype=float32), np.int64(4))


6

In [31]:
model=RNN_Classifier()  ##实例化模型

In [32]:
###定义损失函数和优化器
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),LR)


In [33]:
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter()



In [34]:
##训练模型
for epoch in range(epochs):
    model.train()
    for img,lbl in train_dl:
        optimizer.zero_grad()
        outputs=model(img)
        loss=criterion(outputs,lbl)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0) ###梯度裁剪

        optimizer.step()
    if epoch%10==0:
        print(f"epoch:{epoch} ,loss: {loss.item():.4f}")
    model.eval()
    ##模型评估
    with torch.no_grad():
        correct=0
        total=0
        for img,lbl in test_loader:
            outputs=model((img))
            _,predicted=torch.max(outputs.data,1)
            total+=lbl.size(0)
            correct+=(predicted==lbl).sum().item()
        accuracy=100*correct/total
        if epoch%10==0:
            print(f"epoch:{epoch+1}/{epochs},test accuracy:{accuracy:.2f}")
        writer.add_scalar('test_accracy',accuracy,epoch)
        writer.add_scalar('loss',loss.item(),epoch)
writer.close()
        

epoch:0 ,loss: 3.7114
epoch:1/80,test accuracy:5.00
epoch:10 ,loss: 1.7458
epoch:11/80,test accuracy:42.50
epoch:20 ,loss: 1.0354
epoch:21/80,test accuracy:53.75
epoch:30 ,loss: 0.5747
epoch:31/80,test accuracy:68.75
epoch:40 ,loss: 0.2390
epoch:41/80,test accuracy:83.75
epoch:50 ,loss: 0.1243
epoch:51/80,test accuracy:82.50
epoch:60 ,loss: 0.0696
epoch:61/80,test accuracy:90.00
epoch:70 ,loss: 0.0165
epoch:71/80,test accuracy:90.00


In [35]:
###保存全部
torch.save(model,'rnn_model.pth')

##保存模型参数
torch.save(model.state_dict(),'rnn_model_params.pth')

##加载模型
model_load=RNN_Classifier()
model_load.load_state_dict(torch.load('rnn_model_params.pth'))

C:\Users\ZXD\AppData\Local\Temp\ipykernel_21280\1564482467.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_load.load_state_dict(torch.load('rnn_model_params.pth'))

<All keys matched successfully>